# Multi-Armed Bandits

The most important feature distinguishing reinforcement learning from other types of  
learning is that it uses training information that evaluates the actions taken rather  
than instructs by giving correct actions.

There's multiple type of feedbacks:
- Purely evaluative feedback indicates how good the action taken was, but not whether it was the best or the worst action possible.
- Purely instructive feedback, on the other hand, indicates the correct action to take, independently of the action actually taken.

In this chapter we study the evaluative aspect of reinforcement learning in a simplified  
setting, one that does not involve learning to act in more than one situation. This  
nonassociative setting is the one in which most prior work involving evaluative feedback  
has been done, and it avoids much of the complexity of the full reinforcement learning  
problem.

The particular nonassociative, evaluative feedback problem that we explore is a simple  
version of the k-armed bandit problem.

## A k-armed Bandit Problem

Consider the following learning problem. You are faced repeatedly with a choice among  
k different options, or actions. After each choice you receive a numerical reward chosen  
from a stationary probability distribution that depends on the action you selected.  
Your objective is to maximize the expected total reward over some time period, for example,  
over 1000 action selections, or time steps.

Through repeated action selections you are to maximize your winnings by concentrating your actions on the best levers.

In our k-armed bandit problem, each of the k actions has an expected or mean reward  
given that that action is selected; let us call this the value of that action. We denote the   
action selected on time step t as $A_t$ , and the corresponding reward as $R_t$.  
The value then of an arbitrary action $a$, denoted $q_*(a)$, is the expected reward given that a is selected:

$q_*(a) = E[R_t | A_t = a]$

If you knew the value of each action, then it would be trivial to solve the k-armed bandit  
problem: you would always select the action with highest value. We assume that you do  
not know the action values with certainty, although you may have estimates. We denote  
the estimated value of action $a$ at time step $t$ as $Q_t(a)$.
We would like $Q_t(a)$ to be close to $q_*(a)$.

If you maintain estimates of the action values, then at any time step there is at least  
one action whose estimated value is greatest. We call these the greedy actions. When you  
select one of these actions, we say that you are exploiting your current knowledge of the  
values of the actions. If instead you select one of the nongreedy actions, then we say you  
are exploring, because this enables you to improve your estimate of the nongreedy action’s  
value

### K-armed bandit problem implementation

#### Numpy

For calculation, we're going to use numpy.

#### Gymnasium

To create our environement k-armed bandit, we're gonne use gym,  
which is kind of the standard to create reproductible reinforcement learning environments.

Gym is an open source Python library for developing and comparing reinforcement learning algorithms by providing a standard API to communicate between learning algorithms and environments, as well as a standard set of environments compliant with that API. Since its release, Gym's API has become the field standard for doing this.

Gymnasium is a maintained fork of OpenAI’s Gym library. The Gymnasium interface is simple, pythonic, and capable of representing general RL problems, and has a compatibility wrapper for old Gym environments.

#### Imports

In [1]:
import numpy as np
import gymnasium as gym

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### K-armed Bandit

In [2]:
class KArmedBandit(gym.Env):

    def __init__(self, nb_arms=10, nb_steps=1000, mean=0, variance=1, noise_variance=1):
        self._nb_arms = nb_arms
        self._nb_steps = nb_steps

        self._mean = 0
        self._noise_mean = 0
        self._variance = variance
        self._noise_variance = noise_variance

        self.action_space = gym.spaces.Discrete(nb_arms)
        self.observation_space = gym.spaces.Discrete(1)
    
    def step(self, action):
        self._step += 1
    
        reward = self._arms[action]
        reward_noise = self.np_random.normal(self._noise_mean, self._noise_variance)
        terminated = self._step >= self._nb_steps

        info = { "is_optimal_action": int(action == np.argmax(self._arms)) }

        return reward + reward_noise, terminated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self._step = 0
        self._arms = self.np_random.normal(self._mean, self._variance, size=self._nb_arms)

#### Plotting function

In [3]:
def plot_results(main_title, titles, results):
    fig = make_subplots(rows=1, cols=2, subplot_titles=titles)

    for result_name in results:
        x = np.arange(len(results[result_name]["mean_reward"]))
        fig.add_trace(go.Scatter(x=x, y=results[result_name]["mean_reward"], line_color=results[result_name]["color"], name=result_name), row=1, col=1)
        fig.add_trace(go.Scatter(x=x, y=results[result_name]["optimal_action"], line_color=results[result_name]["color"], name=result_name, showlegend=False), row=1, col=2)

    fig.update_layout(
        title=main_title,
        legend_title="Parameters",
    )

    fig.show()

## Action-value Methods

We begin by looking more closely at methods for estimating the values of actions and
for using the estimates to make action selection decisions, which we collectively call
action-value methods. Recall that the true value of an action is the mean reward when
that action is selected. One natural way to estimate this is by averaging the rewards
actually received:

$$
\begin{aligned}
Q_t(a) &= {sum-of-rewards-when-a-taken-prior-to-t \over number-of-times-a-taken-prior-to-t} \\
       &= {\sum_{i=1}^{t-1} R_i · 1_{A_i = a} \over \sum_{i=1}^{t-1} 1_{A_i=a}}
\end{aligned}
$$

where predicate denotes the random variable that is 1 if predicate is true and 0 if it is not.
If the denominator is zero, then we instead define Q t ( a ) as some default value, such as
0. As the denominator goes to infinity, by the law of large numbers, Q t (a) converges to
q ⇤ ( a ). We call this the sample-average method for estimating action values because each
estimate is an average of the sample of relevant rewards

he simplest action selection rule is to select one of the actions with the highest
estimated value, that is, one of the greedy actions as defined in the previous section.
If there is more than one greedy action, then a selection is made among them in some
arbitrary way, perhaps randomly. We write this greedy action selection method as
A t
.
= argmax
a
Q t (a), 

where argmax a denotes the action a for which the expression that follows is maximized
(again, with ties broken arbitrarily). Greedy action selection always exploits current
knowledge to maximize immediate reward; it spends no time at all sampling apparently
inferior actions to see if they might really be better. A simple alternative is to behave
greedily most of the time, but every once in a while, say with small probability " , instead
28 Chapter 2: Multi-armed Bandits
select randomly from among all the actions with equal probability, independently of
the action-value estimates. We call methods using this near-greedy action selection rule
" -greedy methods. An advantage of these methods is that, in the limit as the number of
steps increases, every action will be sampled an infinite number of times, thus ensuring
that all the Q t ( a ) converge to q ⇤ ( a ). This of course implies that the probability of selecting
the optimal action converges to greater than 1  " , that is, to near certainty. These are
just asymptotic guarantees, however, and say little about the practical e↵ectiveness of
the methods.

#### Action-value Method implementation

In [4]:
class EpsilonGreedy():

    def __init__(self, nb_actions, epsilon):
        self.nb_actions = nb_actions
        self.epsilon = epsilon

        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.ones(self.nb_actions)

    def action(self):
        take_random_action_prob = np.random.uniform(0, 1)

        if take_random_action_prob < self.epsilon:
            return np.random.randint(0, self.nb_actions)
        else:
            return np.argmax(self.q)
    
    def observe(self, action, reward):
        self.nb_action_taken[action] += 1
        self.q[action] += (reward - self.q[action]) / self.nb_action_taken[action]
    
    def reset(self):
        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.ones(self.nb_actions)

## The 10-armed Testbed

To roughly assess the relative e↵ectiveness of the greedy and " -greedy action-value
methods, we compared them numerically on a suite of test problems. This was a set
of 2000 randomly generated k -armed bandit problems with k = 10. For each bandit
problem, such as the one shown in Figure 2.1, the action values, q ⇤ ( a ), a = 1 , . . . , 10,

In [5]:
nb_arms  = 10
nb_steps = 2000

env = KArmedBandit(nb_arms=nb_arms, nb_steps=nb_steps)

#### An example bandit problem from the 10-armed testbed

In [6]:
env.reset()

# Sample our distribution to see it's correct
data = np.array([[env.step(i)[0] for _ in range(2000)] for i in range(len(env._arms))])

fig = go.Figure()

for i in range(len(env._arms)):
    fig.add_trace(go.Violin(x=[i] * len(data[i]), y=data[i], name="q*(" + str(i) + ") = " + str(env._arms[i])[:4], meanline_visible=True))

fig.update_layout(
    title="K-Armed Bandit Problem Distribution",
    xaxis_title="Actions",
    yaxis_title="Reward Distributions",
    legend_title="True Value of q*(a)",
)

fig.show()

In [7]:
def run_env(env, agent):
    list_of_reward = []
    list_of_optimal_action = []

    env.reset()
    agent.reset()

    terminated = False

    while not terminated:
        action = agent.action()

        reward, terminated, info = env.step(action)

        agent.observe(action, reward)

        list_of_reward.append(reward)
        list_of_optimal_action.append(info["is_optimal_action"])
    
    return np.array(list_of_reward), np.array(list_of_optimal_action)

In [8]:
def run_exp(nb_exps, env, agent):
    list_rewards, list_optimal_action = run_env(env, agent)

    for _ in range(nb_exps - 1):
        list_rewards_tmp, list_optimal_action_tmp = run_env(env, agent)

        list_rewards += list_rewards_tmp
        list_optimal_action += list_optimal_action_tmp

    return list_rewards / nb_exps, (list_optimal_action / nb_exps) * 100

In [9]:
nb_exps = 2000

In [10]:
agent_01 = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.01)
mean_rewards_01, percent_optimal_action_01 = run_exp(nb_exps, env, agent_01)

In [11]:
agent_1 = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.1)
mean_rewards_1, percent_optimal_action_1 = run_exp(nb_exps, env, agent_1)

In [12]:
agent_greedy = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.0)
mean_rewards_0, percent_optimal_action_0 = run_exp(nb_exps, env, agent_greedy)

In [13]:
results_exp_1 = {
    'Egreedy 0.01': {
        "mean_reward": mean_rewards_01,
        "optimal_action": percent_optimal_action_01,
        "color": "red"
    },
    'Egreedy 0.1': {
        "mean_reward": mean_rewards_1,
        "optimal_action": percent_optimal_action_1,
        "color": "blue"
    },
    'Greedy 0.0': {
        "mean_reward": mean_rewards_0,
        "optimal_action": percent_optimal_action_0,
        "color": "green"
    },
}

plot_results(
    "Compares greedy method with different parameters (0.01, 0.1 and 0)",
    ["Average Reward / Steps", "Optimal Action / Steps"],
    results_exp_1
)

## Incremental Implementation

In [14]:
class EpsilonGreedy():

    def __init__(self, nb_actions, epsilon):
        self.nb_actions = nb_actions
        self.epsilon = epsilon

        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.ones(self.nb_actions)

    def action(self):
        take_random_action_prob = np.random.uniform(0, 1)

        if take_random_action_prob < self.epsilon:
            return np.random.randint(0, self.nb_actions)
        else:
            return np.argmax(self.q)
    
    def observe(self, action, reward):
        self.nb_action_taken[action] += 1
        self.q[action] += (reward - self.q[action]) / self.nb_action_taken[action]
    
    def reset(self):
        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.ones(self.nb_actions)

2.5 Tracking a Nonstationary Problem

2.6 Optimistic Initial Values

In [ ]:
class EpsilonGreedyInitValue():

    def __init__(self, nb_actions, epsilon=0.1, optimistic_value=0):
        self.nb_actions = nb_actions
        self.epsilon = epsilon
        self.optimistic_value = optimistic_value

        self.nb_times_taken_action = np.zeros(self.nb_actions)
        self.q = np.ones(self.nb_actions) * optimistic_value

    def action(self):
        take_random_action_prob = np.random.uniform(0, 1)

        if take_random_action_prob < self.epsilon:
            return np.random.randint(0, self.nb_actions)
        else:
            return np.argmax(self.q)
    
    def observe(self, action, reward):
        self.nb_times_taken_action[action] += 1
        self.q[action] += 0.1 * (reward - self.q[action]) # / self.nb_times_taken_action[action]
    
    def reset(self):
        self.nb_times_taken_action = np.zeros(self.nb_actions)
        self.q = np.ones(self.nb_actions) * self.optimistic_value


2.7 Upper-Confidence-Bound Action Selection

2.8 Gradient Bandit Algorithms

2.9 Associative Search (Contextual Bandits)

2.10 Summary

parameter study